In [1]:
from __future__ import print_function
from __future__ import division
from future import standard_library
import time
import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.cluster import *
import word2vec

def log_progress(sequence, every=10):
    from ipywidgets import IntProgress
    from IPython.display import display

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record

In [2]:
FILEmodelForInfinitive = Model.load('udpipe_syntagrus.model')
#model = word2vec.load('dataForNeurotext/withOutTypeModel.txt')

In [14]:
%%time
class wordsClass (object):
    model = ''
    vectorOfStr = np.zeros(2000)
    centroids = torch.zeros(2000,300)
    ClasterOfWord = {}
    nomberKnownOfWords = 0
    nomberUNKnownOfWords = 0
    modelForInfinitive = ''

    def loadCentroidsFromTXT (self, source):
        self.centroids = torch.from_numpy(np.loadtxt(source, delimiter=' ', dtype=float))
        self.centroids = torch.reshape(self.centroids, (2000, 300)).cuda()
        
    def saveCentroidsToTXT (self, source):
        f = open (source, 'w')
        for i in range (2000):
            for j in range (300):
                f.write(str(self.centroids[i][j]) + " ")
        f.close()
        
    def saveDict (self, source):
        f = open(source, 'w')
        for k, v in self.ClasterOfWord.items():
            f.write(str(k) + " " + str(v) + "\n")
        f.close()
            
    def loadDict (self, source):
        f = open(source, 'r')
        for strings in f:
            strings = strings.split(' ')
            self.ClasterOfWord.update({strings[0]: int(strings[1])})
        f.close()
        
    def generateCentroids (self, nomberOfClasters, NomberOfIter):
        #scipy.cluster.vq.kmeans2(model.vectors, 2000, iter=100)
        self.centroids = vq.kmeans(self.model.vectors, nomberOfClasters, iter=NomberOfIter)[0]
        
    def generateDict (self, source): #источник = txt для model
        f = open (source, 'r')
        n=0
        words = []
        for line in f:
            line = line.split()
            words.append(line[0])
            n = n+1
        print (words[1])
        f.close()
        for word in log_progress(words, every=100):
            distToCentroids = torch.sum(((self.centroids - self.model[word])**2),1)
            relationID = int(torch.argmin(distToCentroids, 0))
            ClasterOfWord.update({word: relationID})
    
    def distanceBetween2words (self, word1, word2):
        word1 = word1.lower()
        word2 = word2.lower()
        try:
            len = model.distance(tag_ud(text=word1), tag_ud(text=word2))
        except Exception:
            try:
                len =  model.distance(tag_ud(text=word1.capitalize()), tag_ud(text=word2))
            except Exception:
                try:
                    len =  model.distance(tag_ud(text=word1), tag_ud(text=word2.capitalize()))
                except Exception:
                    len = model.distance(tag_ud(text=word1.capitalize()), tag_ud(text=word2.capitalize()))
        return len
    
    def coordinate (self, word):
        word = word.lower()
        try:
            coord = self.model[tag_ud(text=word)]
        except Exception:
            try:
                coord =  self.model[tag_ud(text=word.capitalize())]
            except Exception:
                coord = "error"
        return torch.from_numpy (coord)
    
    def getKey(self, value):
        for k, v in self.ClasterOfWord.items():
            if int(v) == value:
                print (k)
        return k
    
    def addNewWordInClaster (self, newWord, baseWord):
        self.ClasterOfWord.update({newWord: self.ClasterOfWord[baseWord]})
    
    def tryToUpgradeSelfWord2vec (self, word):
        try:
            #print (word)
            relationID = self.ClasterOfWord[word.lower()]
            self.vectorOfStr[relationID] = self.vectorOfStr[relationID] + 1
            self.nomberKnownOfWords = self.nomberKnownOfWords + 1
            #print (relationID)
        except Exception:
            if not re.search(r'[\W]', word):
                #print(self.wordToInf(text=word))
                relationID = self.ClasterOfWord[self.wordToInf(text=word)]
                self.vectorOfStr[relationID] = self.vectorOfStr[relationID] + 1
                self.nomberKnownOfWords = self.nomberKnownOfWords + 1
                #print (relationID)
            else: 
                raise
                
            
    def text2Vec (self, text):
        self.vectorOfStr = np.zeros(2000)
        words = re.findall(r'[0-9A-Za-zА-Яа-я-.]+', text)
        for word in words:
            try :
                self.tryToUpgradeSelfWord2vec(word)
            except Exception:
                for singleWord in re.findall(r"[\w']+", word):
                    try: 
                        self.tryToUpgradeSelfWord2vec(singleWord)
                    except Exception:
                        self.nomberUNKnownOfWords = self.nomberUNKnownOfWords + 1
                        #print ("the word is new: " + singleWord)
        self.vectorOfStr = self.vectorOfStr / self.nomberKnownOfWords    
        return self.vectorOfStr
    
    def wordToInf(self, text):
        process_pipeline = Pipeline(self.modelForInfinitive, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
        wordInfo = process_pipeline.process(text).split('\n')[4].split('\t')
        if (wordInfo[3] == 'NUM'):
            return ('_NUM_' + ('x' * len(wordInfo[2])))
        else:
            return wordInfo[2]
        
#проработать встречи ::
#реализовать работу с реляционной БД (?):
#(текст->оценка принадлежности вектор->оценка принадлежности)
#поддержка новый слов через сервис

wordsClass = wordsClass()
wordsClass.modelForInfinitive = FILEmodelForInfinitive
wordsClass.loadDict('correct_dictionary.txt')

#wordsClass.text2Vec("рaссказывать")
#wordsClass.ClasterOfWord['Alibaba']
#wordsClass.saveDict('dict words has clasters.txt')

#wordsClass.text2Vec("Китайский интернет-гигант Alibaba меняет бизнес-модель торговой площадки AliExpress, дав возможность продавцам из России, а также Турции, Италии и Испании продавать на ней свои товары. Об этом сообщает «Интерфакс».")
print (wordsClass.vectorOfStr)

[0. 0. 0. ... 0. 0. 0.]
CPU times: user 208 ms, sys: 11.5 ms, total: 220 ms
Wall time: 220 ms


In [17]:
%%time
#обрабатываем все данные
Ntext = 0
NKnownWods = 0
NUNKnownWords = 0
f = open ('dataWithText.txt', 'r')
fout = open ('data.txt' , 'w')
relationToClasses = np.zeros(42)

sumOfVectors = np.zeros(2000)

NForVisualization = 0

for strin in f:
    strs = strin.split('~')
    
    vecOfText = wordsClass.text2Vec(strs[1])
    #получили вектор текста
    
    text = ''
    for k in range (2000):
        text = text + str(vecOfText[k])
        if k != 1999:
            text = text + '~'
    #сформировали строку-вектор текста
    
    for l in range (2, 44):
        relationToClasses[l-2] = float (strs[l])
    #загрузили вектор принадлежности данного текста
    
    vecToStr = ''
    for k in range (42):
        vecToStr = vecToStr + str(relationToClasses[k])
        if k != 41:
            vecToStr = vecToStr + '~'
    #сформировали строку-вектор принадлежности
    
    fout.write(text + '|' + vecToStr + '\n')
    #записали
    
    sumOfVectors = sumOfVectors + vecOfText
    NKnownWods = NKnownWods + wordsClass.nomberKnownOfWords
    NUNKnownWords = NUNKnownWords + wordsClass.nomberUNKnownOfWords
    Ntext = Ntext + 1
    
    if (Ntext > NForVisualization + 20):
        NForVisualization = Ntext
        print (NForVisualization) 
    
f.close()
fout.close()
print ('понято слов: ' + str(NKnownWods))
print ('не понято слов: ' + str(NUNKnownWords))
print ('всего текстов обработано: ' + str(Ntext))

21
42
63
84
105
126
147
168
189
210
231
252
273
294
315
336
357
378
399
420
441
462
483
504
525
546
567
588
609
630
651
672
693
714
735
756
777
798
819
840
861
882
903
924
945
966
987
1008
1029
1050
1071
1092
1113
1134
1155
1176
1197
1218
1239
1260
1281
1302
1323
1344
1365
1386
1407
1428
1449
1470
1491
1512
1533
1554
1575
1596
1617
1638
1659
1680
1701
1722
1743
1764
1785
1806
1827
1848
1869
1890
1911
1932
1953
1974
1995
2016
2037
2058
2079
2100
2121
2142
2163
2184
2205
2226
2247
2268
2289
2310
2331
2352
2373
2394
2415
2436
2457
2478
2499
2520
2541
2562
2583
2604
2625
2646
2667
2688
2709
2730
2751
2772
2793
2814
2835
2856
2877
2898
2919
2940
2961
2982
3003
3024
3045
3066
3087
3108
3129
3150
3171
3192
3213
3234
3255
3276
3297
3318
3339
3360
3381
3402
3423
3444
3465
3486
3507
3528
3549
3570
3591
3612
3633
3654
3675
3696
3717
3738
3759
3780
3801
3822
3843
3864
3885
3906
3927
3948
3969
3990
4011
понято слов: 5235375884
не понято слов: 851772535
всего текстов обработано: 4022
CPU times: user